In [21]:
import pandas as pd
import Preprocessing_and_encoding as preprocess
import Randomiser as rr
import Budget_division as bd
import pandas as pd
import numpy as np
import random
import Helper as hp
import datetime
import time

## Main configuration

In [22]:
# Input: appliance (n) list with their energy consumption values
appliance_list = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']
n = len(appliance_list) # Number of appliances
# Input: granularity levels with ranges(d-levels)
domain = ['l10', 'l9', 'l8', 'l7', 'l6', 'l5', 'l4', 'l3', 'l2', 'l1' ]
granularity_levels = {
  "l1": "0-250",
  "l2": "250-500",
  "l3": "500-750",
  "l4": "750-1000",
  "l5": "1000-1250",
  "l6": "1250-1500",
  "l7": "1500-1750",
  "l8": "1750-2000",
  "l9": "2000-2500",
  "l10": "2500"
}
applaince_energy_list = []
appliance_energy_dict = {}
#Tuning parameters
epsilon = 5 # 0 < ε ≤ 10
alpha = 10 # 4-10
window_size = 6
window_appliance_data = []
window_binary_array_data = []

## Data loading

In [23]:
A1 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/air_purifier_293.csv")
A2 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/air_conditioner_222.csv")
A3 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/computer_44.csv")
A4 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/dehumidifier_310.csv")
A5 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/fan_215.csv")
A6 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/freezer_249.csv")
A7 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/fridge_284.csv")
A8 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/internet_router_131.csv")
A9 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/laptop_64.csv")
A10 = pd.read_csv("/home/z5288879/Documents/DifferentialPrivacy/Data/KaggleApplinaceData/printer_286.csv")

In [24]:
appliance_list = [A1, A2, A3, A4, A5, A6, A7, A8, A9, A10]

In [25]:
#Calculating energy per day as we send the data once a day
for i in range(len(appliance_list)):
    consumption_per_day = preprocess.energy_per_day_calculation(appliance_list[i])
    var_name = 'appliance_energy{}'.format(i)
    # Assign a value to the variable
    globals()[var_name] = []

    for j in range(20):
        globals()[var_name].append(consumption_per_day['power'][j])  # Energy consumption for an appliance in a day
    applaince_energy_list.append(globals()[var_name])  # Energy consumption for 7 days for all the appliances

In [28]:
# Appliance_energy dictionary per day for seven days. In appliance_energy_dict_days, key specifies the day
# (day 1, 2, 3...), and value represents the dictionary of  appliances with their energy consumption for the day.
appliance_energy_dict_days = {}
for j in range (20):
    var_name = 'appliance_energy_dict_{}'.format(j)
    globals()[var_name] = {}
    for i in range(len(appliance_list)):
        globals()[var_name]["A"+str(i+1)]= applaince_energy_list[i][j]
    appliance_energy_dict_days["day"+str(j+1)]= globals()[var_name]

In [29]:
appliance_energy_dict_days

{'day1': {'A1': 154.76,
  'A2': 0.0,
  'A3': 1697.86,
  'A4': 11416.97,
  'A5': 492.0,
  'A6': 1420.29,
  'A7': 2361.45,
  'A8': 334.8,
  'A9': 142.47,
  'A10': 236.78},
 'day2': {'A1': 87.48,
  'A2': 0.0,
  'A3': 2693.55,
  'A4': 21651.82,
  'A5': 81.21,
  'A6': 3531.4,
  'A7': 2375.24,
  'A8': 752.69,
  'A9': 519.84,
  'A10': 254.4},
 'day3': {'A1': 87.55,
  'A2': 0.0,
  'A3': 8449.29,
  'A4': 22555.2,
  'A5': 85.57,
  'A6': 3680.78,
  'A7': 2460.09,
  'A8': 760.74,
  'A9': 556.73,
  'A10': 254.37},
 'day4': {'A1': 104.36,
  'A2': 0.0,
  'A3': 9502.41,
  'A4': 21785.13,
  'A5': 1241.93,
  'A6': 3579.65,
  'A7': 2122.31,
  'A8': 759.24,
  'A9': 404.91,
  'A10': 288.35},
 'day5': {'A1': 265.14,
  'A2': 0.0,
  'A3': 10953.57,
  'A4': 14913.48,
  'A5': 2986.22,
  'A6': 3237.8,
  'A7': 2398.95,
  'A8': 762.72,
  'A9': 2.52,
  'A10': 286.33},
 'day6': {'A1': 253.04,
  'A2': 337.91,
  'A3': 9934.91,
  'A4': 5717.84,
  'A5': 1343.6,
  'A6': 2905.69,
  'A7': 2095.19,
  'A8': 763.28,
  'A9': 6

In [30]:
#Encode the energy data into energy levels (granularity levels)
for key, value in appliance_energy_dict_days.items():
    true_energy_levels_dict = preprocess.create_appliance_energy_encoding(appliance_energy_dict_days[key], granularity_levels)
    # Declare, l = (d*n); // d- number of granular levels, n- number of appliances
    # Generate a long binary string B with the length of l by merging the binary arrays
    binary_array = []
    for i in range(len(true_energy_levels_dict)):
        binary_array.append(list(true_energy_levels_dict.values())[i])
    binary_array = np.array(binary_array).flatten()

    window_appliance_data.append(true_energy_levels_dict.values())
    window_binary_array_data.append(binary_array)

In [32]:
# Adding the encoded data in a sliding window format for our calculation
for window in hp.sliding_window(window_binary_array_data, window_size):
    current_window = window
    print(current_window)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0,

In [34]:
current_window

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 

## Budget divis